In [6]:
spark.stop()

In [2]:
import pyspark
import json
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpt
from pyspark.sql import SparkSession
from datetime import datetime, date
from pyspark.sql import Row
from graphframes import *
from delta import *
from delta.tables import *

from pyspark.sql.functions import col, hex, base64, avg, collect_list, concat, lit, max
from pyspark.sql.types import StringType,BooleanType,DateType

In [4]:
config = pyspark.SparkConf().setAll([
    ('spark.executor.memory', '12g'), 
    ('spark.executor.cores', '3'), 
    ('spark.cores.max', '3'),
    ('spark.driver.memory','1g'),
    ('spark.executor.instances', '1'),
    ('spark.dynamicAllocation.enabled', 'true'),
    ('spark.dynamicAllocation.shuffleTracking.enabled', 'true'),
    ('spark.dynamicAllocation.executorIdleTimeout', '60s'),
    ('spark.dynamicAllocation.minExecutors', '0'),
    ('spark.dynamicAllocation.maxExecutors', '2'),
    ('spark.dynamicAllocation.initialExecutors', '1'),
    ('spark.dynamicAllocation.executorAllocationRatio', '1')
])

In [5]:
# use for 8889

builder = pyspark.sql.SparkSession.builder \
    .appName("Neo4j Testing") \
    .master("spark://172.23.149.212:7077") \
    .config(conf=config) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/05/05 14:09:04 WARN Utils: Your hostname, algorand-druid-and-spark resolves to a loopback address: 127.0.0.1; using 172.23.149.212 instead (on interface ens3)
22/05/05 14:09:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ff512511-229e-4a83-a296-d619718b3546;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 143ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      d

In [ ]:
#use for 8890

spark = SparkSession \
    .builder \
    .config(conf=config) \
    .master("spark://172.23.149.212:7077") \
    .appName("Neo4j Example") \
    .getOrCreate()


In [48]:
# read from neo4j database
# how can i change the dabatase to be read from? is it the one that neo4j uses at the moment?
# make sure the movie graph is loaded in neo4j, otherwise no data will be shown

dfPerson = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("authentication.basic.username", "neo4j") \
  .option("authentication.basic.password", "password") \
  .option("labels", "Person") \
  .load()

dfPerson.where("born > 1990").show()

+----+--------+--------------------+----+
|<id>|<labels>|                name|born|
+----+--------+--------------------+----+
|  44|[Person]|   Jonathan Lipnicki|1996|
| 173|[Person]|Janosch Baltenspe...|1995|
| 175|[Person]|      Domenic Fuerer|1998|
| 177|[Person]|        Lennart Jung|1998|
+----+--------+--------------------+----+



22/05/05 14:05:29 WARN SchemaService: Switching to query schema resolution


In [30]:
dfPerson.show()

+----+--------+------------------+----+
|<id>|<labels>|              name|born|
+----+--------+------------------+----+
|   1|[Person]|      Keanu Reeves|1964|
|   2|[Person]|  Carrie-Anne Moss|1967|
|   3|[Person]|Laurence Fishburne|1961|
|   4|[Person]|      Hugo Weaving|1960|
|   5|[Person]|   Lilly Wachowski|1967|
|   6|[Person]|    Lana Wachowski|1965|
|   7|[Person]|       Joel Silver|1952|
|   8|[Person]|       Emil Eifrem|1978|
|  12|[Person]|   Charlize Theron|1975|
|  13|[Person]|         Al Pacino|1940|
|  14|[Person]|   Taylor Hackford|1944|
|  16|[Person]|        Tom Cruise|1962|
|  17|[Person]|    Jack Nicholson|1937|
|  18|[Person]|        Demi Moore|1962|
|  19|[Person]|       Kevin Bacon|1958|
|  20|[Person]| Kiefer Sutherland|1966|
|  21|[Person]|         Noah Wyle|1971|
|  22|[Person]|  Cuba Gooding Jr.|1968|
|  23|[Person]|      Kevin Pollak|1957|
|  24|[Person]|        J.T. Walsh|1943|
+----+--------+------------------+----+
only showing top 20 rows



In [6]:
dfWrite = spark.createDataFrame([
    Row(name="Lennart Jung", born=1998),
    Row(name="Domenic Fuerer", born=1998),
    Row(name="Janosch Baltensperger", born=1995)
])
dfWrite.show()

+--------------------+----+
|                name|born|
+--------------------+----+
|        Lennart Jung|1998|
|      Domenic Fuerer|1998|
|Janosch Baltenspe...|1995|
+--------------------+----+



In [46]:
dfWrite.write.format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("labels", ":Person") \
  .option("node.keys", "name") \
  .save()

22/05/05 14:05:24 WARN SchemaService: Switching to query schema resolution
